# HPP with multiple energy markets

## Evaluating the performance of a hybrid power plant using HyDesign

HyDesign is an open-source tool for design and optimization of utility scale wind-solar-storage based hybrid power plants.

In this notebook we will evaluate a hybrid power plant design in a specific location.

A hybrid power plant design consists on selecting the following parameters:

**Wind Turbine design:**

1. Clearance [m] (`clearance`): Height from the ground to rotor tip at lowest posstion. This parameter controls the wind turbine hub height given a rotor radius: `clearance =  HH - R`. 
2. Specific power of the wind turbine [MW/m2] (`sp`): Defined as `sp = p_rated / ( pi * R^2 )`, it controls the turbine rotor size for a given rated power. Turbines with lower specific power produce more power at lower wind speeds, but are more expensive.
3. Rated power of the wind turbine [MW] (`p_rated`)

**Wind Plant design:**

4. Number of wind turbines in the wind plant [-] (`Nwt`)
5. Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

6. Solar plant power capacity [MW] (`solar_MW`)
7. Surface tilt [deg] (`surface_tilt`)
8. Surface azimuth [deg] (`surface_azimuth`)
9. DC-AC ratio [-] (`solar_DCAC`): This parameter controls how much over-planting of PV (in DC power) is connected to the inverters. It is common practice in PV design to have `solar_DCAC = 1.5`. 

**Battery Storage design:**

10. Battery power [MW] (`b_P`)
11. Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
12. Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


##
**Imports**

Install hydesign if needed.
Import basic libraries. 
Import HPP model assembly class.
Import the examples file path.

In [ ]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git    

In [ ]:
import os
import time
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from hydesign.assembly.hpp_assembly_BM import hpp_model
from hydesign.examples import examples_filepath

##
**Specifying the site**

Hydesign, provides example data from several sites in India and Europe. 

The site coordinates (longitude, latitude, and altitude) are given in `examples_sites.csv`.

In [ ]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0, sep=';')
examples_sites

In [ ]:
name = 'Denmark_good_wind_BM'
ex_site = examples_sites.loc[examples_sites.name == name]

longitude = ex_site['longitude'].values[0]
latitude = ex_site['latitude'].values[0]
altitude = ex_site['altitude'].values[0]

input_ts_fn = examples_filepath+ex_site['input_ts_fn'].values[0]
sim_pars_fn = examples_filepath+ex_site['sim_pars_fn'].values[0]
input_HA_ts_fn = examples_filepath+ex_site['input_HA_ts_fn'].values[0]
price_up_ts_fn = examples_filepath+ex_site['price_up_ts'].values[0]
price_dwn_ts_fn = examples_filepath+ex_site['price_dwn_ts'].values[0]
price_col = ex_site['price_col'].values[0]

In [ ]:
weather_HA = pd.read_csv(input_HA_ts_fn, index_col=0, parse_dates=True)
SO_imbalance = weather_HA['SO_power_imbalance']

## 
**Initializing the HPP model**

Initialize the HPP model (hpp_model class) with the coordinates and the necessary input files.

In [ ]:
hpp = hpp_model(
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        num_batteries = 5,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
        input_HA_ts_fn = input_HA_ts_fn,
        price_up_ts_fn = price_up_ts_fn,
        price_dwn_ts_fn = price_dwn_ts_fn,
        price_col = price_col,
)

##
### Evaluating the HPP model

In [ ]:
start = time.time()

clearance = 10
sp = 350
p_rated = 5
Nwt = 70
wind_MW_per_km2 = 7
solar_MW = 0
surface_tilt = 50
surface_azimuth = 180
solar_DCAC = 1.5
b_P = 100
b_E_h  = 3
cost_of_batt_degr = 2

x = [clearance, sp, p_rated, Nwt, wind_MW_per_km2, \
solar_MW, surface_tilt, surface_azimuth, solar_DCAC, \
b_P, b_E_h , cost_of_batt_degr]

outs = hpp.evaluate(*x)

hpp.print_design()

end = time.time()
print(f'exec. time [min]:', (end - start)/60)

In [ ]:
b_E_SOC_t = hpp.prob.get_val('ems.b_E_SOC_t')
b_E_SOC_BM_t = hpp.prob.get_val('ems.b_E_SOC_BM_t')
b_SM_t = hpp.prob.get_val('ems.b_t')
b_BM_t = hpp.prob.get_val('ems.b_BM_t')
price_t = hpp.prob.get_val('ems.price_t')
price_up_reg_t = hpp.prob.get_val('ems.price_up_reg_t')
price_dwn_reg_t = hpp.prob.get_val('ems.price_dwn_reg_t')

wind_t = hpp.prob.get_val('ems.wind_t')
wind_BM_t_ext = hpp.prob.get_val('ems.wind_BM_t_ext')
SO_imbalance_t = hpp.prob.get_val('ems.SO_imbalance_t')
hpp_t = hpp.prob.get_val('ems.hpp_t')
hpp_BM_t = hpp.prob.get_val('ems.hpp_BM_t')
hpp_curt_SM_t = hpp.prob.get_val('ems.hpp_curt_t')
hpp_curt_BM_t = hpp.prob.get_val('ems.hpp_curt_BM_t')
price_penalty_BM_t = hpp.prob.get_val('ems.price_penalty_BM_t')
P_hpp_up_t = hpp.prob.get_val('ems.P_hpp_up_t')
P_hpp_dwn_t = hpp.prob.get_val('ems.P_hpp_dwn_t')
P_hpp_up_max_t = hpp.prob.get_val('ems.P_hpp_up_max_t')
P_hpp_dwn_max_t = hpp.prob.get_val('ems.P_hpp_dwn_max_t')
grid_MW = hpp.prob.get_val('ems.G_MW')
hpp_curt_t = hpp_curt_BM_t


In [ ]:
# Create a figure and axis
light_yellow = (1.0, 1.0, 0.2)  # (R, G, B)       
fig, ax = plt.subplots(figsize=(18, 6))
n_days_plot = 2
sd = 0
# Plot wind_DA as a bar and wind_HA as a narrow bar
ax.bar(np.arange(len(wind_t[sd:24*n_days_plot])), wind_t[sd:24*n_days_plot], color='white',width=0.8, hatch='///', edgecolor='grey', label='Wind DA')
ax.bar(np.arange(len(wind_BM_t_ext[sd:24*n_days_plot])), wind_BM_t_ext[sd:24*n_days_plot], color='white', width=0.3, hatch='..', edgecolor=light_yellow, label='Wind HA')

# Line plot for HPP power and curtailment
plt.plot(hpp_t[sd:24*n_days_plot], color='purple', linewidth='1.5', label ='HPP power')
plt.plot(hpp_curt_SM_t[sd:24*n_days_plot], color='black', linewidth='1', label ='Curtailment - DA')
plt.plot(hpp_curt_t[sd:24*n_days_plot], color='black', linestyle='--', linewidth='1.5', label ='Curtailment')
plt.plot(b_BM_t[sd:24*n_days_plot], color='blue', linewidth='1.5', linestyle='-.', label ='Battery power - BM')
plt.plot(b_E_SOC_BM_t[sd:24*n_days_plot], color='darkblue', linestyle=':', linewidth='1.5', label ='Battery SoC - BM')


# Plot Up and down regulation power as a narrow bar with a circle marker at the top
ax.bar(np.arange(len(P_hpp_up_t[sd:24*n_days_plot])), P_hpp_up_t[sd:24*n_days_plot], color='green', width=0.1, align='center', label='Up reg')
ax.scatter(np.arange(len(P_hpp_up_t[sd:24*n_days_plot])), P_hpp_up_t[sd:24*n_days_plot], color='green', zorder=3)

ax.bar(np.arange(len(P_hpp_dwn_t[sd:24*n_days_plot])), P_hpp_dwn_t[sd:24*n_days_plot], color='orange', width=0.1, align='center', label='Down reg')
ax.scatter(np.arange(len(P_hpp_dwn_t[sd:24*n_days_plot])), P_hpp_dwn_t[sd:24*n_days_plot], color='orange', zorder=3)

# # Customize ticks and labels
ax.set_xticks(np.arange(0, 24*n_days_plot))
plt.yticks(np.arange(-30, 361, step=30))
plt.xticks(fontsize=12, fontname='Times New Roman')  # Set font size for x-axis ticks
plt.yticks(fontsize=12, fontname='Times New Roman')  # Set font size for y-axis ticks
plt.xlabel('Time (hours)', fontsize=12, fontname='Times New Roman')  # Set label for x-axis
plt.ylabel('Power (MW)', fontsize=12, fontname='Times New Roman')  # Set label for y-axis
# Add legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.19),
           ncol=3, fancybox=0, shadow=0, fontsize=10)
plt.show()

In [ ]:
results = {'Wind SM': wind_t, 'Wind BM': wind_BM_t_ext[:8760], 'HPP power output SM': hpp_t[:8760],'HPP power output BM': hpp_BM_t[:8760], 'Curtailment SM': hpp_curt_SM_t[:8760], 'Curtailment BM': hpp_curt_BM_t[:8760], \
                    'Battery power - SM': b_SM_t[:8760], 'Battery power - BM': b_BM_t[:8760], \
                        'Energy Level': b_E_SOC_t[:8760], 'Up regulation power': P_hpp_up_t[:8760], 'Down regulation power': P_hpp_dwn_t[:8760],\
                            'Max Up regulation': P_hpp_up_max_t[:8760] , 'Max Down regulation': P_hpp_dwn_max_t[:8760] , 'Penalty BM': price_penalty_BM_t[:8760],\
                                 'SM Prices': price_t, 'Up reg prices': price_up_reg_t, 'Down reg prices': price_dwn_reg_t}
df = pd.DataFrame(results)
df.to_csv('EMS_out_9_BM.csv')


In [ ]:
design_df = hpp.evaluation_in_df()
design_df.to_csv('output_9_BM.csv')